# This notebook is calculate accuaracy of CIFAR 10 quick with NVDLA

## Environment

* NVDLA 
    * Hardware: nvdla/nvdlav1
    * Software: nvdla/sw @github 06/01/2018
* Model
    * Dataset: 10000 images of CIFAR10 test  
    * Model: CIFAR 10 quick

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os
import caffe
import time
import glob

In [2]:
def top_k(result, top_num, label_arr):
    tmp = -(top_num+1)
    top_k = result.flatten().argsort()[-1:tmp:-1]
    top_num = 1
    for i in np.arange(top_k.size):
        print("Top %d:\tProb:%2.4f%%\t[%d]\t %s"%(top_num,result[top_k[i]]*100,top_k[i], labels[top_k[i]]))
        top_num = top_num + 1

In [3]:
def read_files(path,fileType):
    dimg_files = []
    for root, directories, filenames in os.walk(path):
        for filename in filenames:
            if fileType in filename:
                dimg_files.append(os.path.join(root,filename))
    return dimg_files

In [4]:
def read_dimg(dimg_file):
    return np.genfromtxt(dimg_file, float, delimiter=' ')

In [5]:
dimg_root_path = '/home/caslab/workspace/NVDLA/sw_6_1/prebuilt/linux/runtime/test/'
dimg_files = read_files(dimg_root_path,'dimg')



test_name = []
nvdla_result = []

for element in dimg_files:
    tmp = os.path.basename(element).split('.')[0].split('_')[1]
    test_name.append(tmp)
    nvdla_result.append(read_dimg(element))
    


In [6]:
cifar10_synset_file = '/home/caslab/workspace/dataset/cifar/labels.txt'
cifar_labels = np.genfromtxt(cifar10_synset_file, str, delimiter='\n')

In [7]:
show = False
if show:
    for i in range(0,len(nvdla_result)):
        plt.rcdefaults()
        fig, ax = plt.subplots()

        label = ("airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck")
        y_pos = np.arange(len(label))

        ax.barh(y_pos, nvdla_result[i], align='center',
                color='gray', ecolor='black')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(label)
        plt.xlabel("Class")
        ax.invert_yaxis()  # labels read top-to-bottom
        ax.set_xlabel('Probability')
        ax.set_title('CIFAR10 Quick Classification Ans:%s'%test_name[i])

        plt.show()

In [8]:
counter = 0.0
for i in range(0,len(nvdla_result)):
    if (cifar_labels[nvdla_result[i].argmax()] == test_name[i]):
        counter += 1

In [9]:
print("Accuracy: %.3f%%, Totol test num: %d"%((counter*100/len(nvdla_result)),len(nvdla_result)))

Accuracy: 25.378%, Totol test num: 2380
